# 前后设计模式的不利之处
如果之前您做过无操作系统的嵌入式开发，使用“前台main——后台中断”这种前后台模式开发。

例如，任务1，你要完成接收串口来的1000字节数据，接收完毕数据后，进行校验，校验正确后，把所有数据乘以100，再从串口将数据发送(暂不考虑异常时处理)；任务2，你要完成从网口接收1000字节数据，进行校验，校验正确后，把所有数据乘以100，再从网口将数据发送(暂不考虑异常时处理)。

对于上述要求的任务，常规逻辑就是：在串口中断中，将接收到的数据放入数据缓冲区中，当接收完毕1000字节后，置串口数据接收完毕标志，程序代码main中的while(1)循环中查询串口接收标志有效，怎进行数据的校验和乘以100的处理；同理，网络接收数据也是这样处理，中断中接收数据、置标志，主循环中数据处理。C语言伪代码如下：

//串口中断处理函数

void uart_irq(void){

    //从串口寄存器中读数据，并放在uart_data[]缓冲区中
    uart_data[uart_data_len] = uart_rev_register;
    uart_data_len ++;
    
    //接收到UART_REV_LEN(当前要求为1000字节)，则置标志有效
    if(uart_data_len >= UART_REV_LEN){
        uart_rev_flag = TRUE;
        uart_data_len = 0;
    }
}

//网口中断处理函数

void net_irq(void){

    //从网口寄存器中读取数据，并放在net_data缓冲区中
    for(temp_i = 0; temp_i < net_len_reg; temp_i ++){
        net_data[net_data_len] = net_data_register;
        net_data_len ++;
    }
    
    //则置标志有效 
    net_rev_flag = TRUE;
    uart_data_len = 0;
}

void main(void){

    while(1){
    
        //串口接收标志有效
        if(uart_rev_flag == TRUE){
        
            //置标志为无效，用于下次数据接受
            uart_rev_flag == FALSE;
            
            //调用串口数据处理函数
            uart_data_process_func();
        }

        //网口接收标志有效
        if(net_rev_flag == TRUE){
        
            //置标志为无效，用于下次数据接受
            net_rev_flag == FALSE;
            
            //调用网口数据处理函数
            net_data_process_func();
        }
    }
}

如果你的项目要求20个任务，那你的main会很庞大，但也能做；如果，部分任务之前有任务的紧急程度不同，那用这种“前后模式”设计程序，非常考研设计的任务规划能力，再加上后续用户不断的新增新的任务，那经过若干次修改后，程序简直不忍直视。

# 忠告：
不要小看了用户修改，我有个项目，历经5年时间，修改了无数的版本；要记住一句忠告“用户虐我千百遍，我待用户如初恋”，用户永远是上帝。